In [1]:
import os
from pathlib import Path
import pandas as pd

import requests

import fiona
import shapely

import sys
from pathlib import Path
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
import library.validation.accuracy.metric as metric
import library.validation.accuracy.complexity as complexity

# read data

In [2]:
data_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/data_sites"
terrain_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/terrain_complexity/dragut_eisank_2012"


In [3]:
sites_metadata = pd.read_csv(os.path.join(data_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


# terrain classification

In [4]:
sites_metadata['terrain_classification'] = None
with fiona.open(os.path.join(terrain_path, 'Level3.shp')) as fiona_collection:
    for shapefile_record in fiona_collection:
        shape = shapely.geometry.shape(shapefile_record['geometry'])
        
        for index, row in sites_metadata.iterrows():
            point = shapely.geometry.Point(row['longitude'], row['latitude']) 
            
            if shape.contains(point):
                terrain_classification = shapefile_record['properties']['Class_name']                
                sites_metadata.loc[index, 'terrain_classification'] = terrain_classification
                break

In [5]:
sites_metadata

,country,latitude,longitude,altitude,provider,terrain_classification
station_name,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,_flat plains
carpentras,France,44.08,5.06,100.0,bsrn,_flat plains
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,_flat plains
magurele_mars,Romania,44.34,26.01,110.0,bsrn,_flat plains
cabauw,Netherlands,51.97,4.93,0.0,bsrn,_flat plains
...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,_flat plains
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,_flat plains
gonzaga,Italy,44.96,10.77,16.0,lombardy,_flat plains


In [6]:
sites_metadata['terrain_classification'].unique()

array(['_flat plains', '_irregular plains', '_smooth low hills', None,
       '_high hills', '_tablelands', '_low mountains', '_rough low hills'],
      dtype=object)